<a href="https://colab.research.google.com/github/nedokormysh/GB_DA_algorithms/blob/lesson8/DA_algorithms_hw_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import datasets

# Задание 1.

Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

## PCA с занятия.

In [2]:
def accuracy(y, y_pred):
  return np.sum(y == y_pred) / len(y)

def e_metrics(x1, x2):
    distance = np.sum(np.square(x1 - x2))
    return np.sqrt(distance)

def standart_scale(X):
  X_ = X.astype(float)
  return X - X.mean(axis=0) / np.std(X_, axis=0) 

In [3]:
# knn с прошлого занятия
def knn(x_train, y_train, x_test, k, weights=False, q = 0.5):
    
    answers = []
    for x in x_test:
        test_distances = [] # обнуляем список вычесленных расстояний.


        #print(len(answers))    
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
                        
            # Записываем в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}

        # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов
        for d in sorted(test_distances)[0:k]:
          if weights == True:
            weight = q ** d[0]
            classes[d[1]] += weight
          else:
            classes[d[1]] += 1
            
        # Записываем в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
    return answers

In [4]:
# соберём pca
def pca(X, component=1, visual=False):
  # Найдем собственные векторы и собственные значения
  covariance_matrix = X.T.dot(X)
  eig_values, eig_vectors = np.linalg.eig(covariance_matrix)
  
  # сформируем список кортежей (собственное значение, собственный вектор)
  eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

  # и отсортируем список по убыванию собственных значений
  eig_pairs.sort(key=lambda x: x[0], reverse=True)

  eig_sum = sum(eig_values)
  var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
  cum_var_exp = np.cumsum(var_exp)
  
  # Сформируем вектор весов из собственных векторов
  W = np.hstack([eig_pairs[i][1].reshape(len(eig_pairs[i][1]),1) for i in range(component)])

  # Сформируем новую матрицу "объекты-признаки"
  Z = X.dot(W)

  if visual:
    print('Собственные значения в порядке убывания:')
    for i in eig_pairs:
      print(i)

    print('-'*50) 

    print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

    # а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
    print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

    print('-'*50)  
    print(f'Матрица весов W:\n', W)

  return Z

In [5]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X_iris = iris.data
y_iris = iris.target
X_iris.shape, y_iris.shape

((150, 4), (150,))

In [6]:
# стандартизируем признаки

X_iris_st = standart_scale(X_iris)

In [7]:
X_train_iris, X_test_iris, Y_train_iris, Y_test_iris = train_test_split(X_iris_st, y_iris,
                                                    test_size=0.3,
                                                    random_state=24,
                                                    stratify=y_iris)
X_train_iris.shape, X_test_iris.shape

((105, 4), (45, 4))

In [8]:
y_pred = knn(X_train_iris, Y_train_iris, X_test_iris, 5, weights=True)

In [9]:
# Применим алгоритм уменьшения размерности PCA

X_pca = pca(X_iris_st, component=2)
# X_pca[:10]

In [10]:
# Снова разобьем данные

X_train_iris_pca, X_test_iris_pca, Y_train_iris_pca, Y_test_iris_pca = train_test_split(X_pca, y_iris,
                                                            test_size=0.3,
                                                            random_state=24,
                                                            stratify=y_iris)
X_train_iris_pca.shape, X_test_iris_pca.shape

((105, 2), (45, 2))

In [11]:
# Обучим модель KNN после применения PCA
y_pred_pca = knn(X_train_iris_pca, Y_train_iris_pca, X_test_iris_pca, 5, weights=True)

In [12]:
print(f'Модель KNN до применения PCA, accuracy: {accuracy(Y_test_iris, y_pred):.4f}', 
      f'Модель KNN после применения PCA, accuracy: {accuracy(Y_test_iris_pca, y_pred_pca):.4f}', sep='\n')

Модель KNN до применения PCA, accuracy: 0.9556
Модель KNN после применения PCA, accuracy: 0.9111


## PCA встроенный в sklearn

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

In [14]:
from sklearn.datasets import load_digits
import seaborn as sns

digits = load_digits()
X_load_digits = digits.data
y_load_digits = digits.target
X_load_digits.shape, y_load_digits.shape

((1797, 64), (1797,))

In [15]:
X_train_load_digits, X_test_load_digits, Y_train_load_digits, Y_test_load_digits = train_test_split(X_load_digits, y_load_digits,
                                                            test_size=0.3,
                                                            random_state=24,
                                                            stratify=y_load_digits)
X_train_load_digits.shape, X_test_load_digits.shape

((1257, 64), (540, 64))

In [20]:
cls = LogisticRegression(solver='liblinear').fit(X_train_load_digits, Y_train_load_digits)


tPCA = PCA(n_components=10)
tPCA.fit(X_train_load_digits)
X_test_pca_load_digits = tPCA.transform(X_test_load_digits)

cls_pca = LogisticRegression(solver='liblinear').fit(tPCA.transform(X_train_load_digits), Y_train_load_digits)

y_p_prob = cls.predict_proba(X_test_load_digits)#[:,1]
y_p_pca_prob = cls_pca.predict_proba(X_test_pca_load_digits)#[:,1]

y_p = cls.predict(X_test_load_digits)#[:,1]
y_p_pca = cls_pca.predict(X_test_pca_load_digits)#[:,1]

In [17]:
from sklearn.metrics import RocCurveDisplay, roc_curve, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt

In [18]:
print(f'Модель LogisticRegression до применения PCA, accuracy: {accuracy(Y_test_load_digits, y_p)}', 
      f'Модель LogisticRegression после применения PCA, accuracy: {accuracy(Y_test_load_digits, y_p_pca)}', sep='\n')

Модель LogisticRegression до применения PCA, accuracy: 0.9555555555555556
Модель LogisticRegression после применения PCA, accuracy: 0.9222222222222223


При применении PCA у меня в обоих случаях (разные датасеты и разные модели) выше точность уменьшилась относительно использования применения моделей без понижения размерности.

In [21]:
print(f'linear roc_auc_score = {roc_auc_score(Y_test_load_digits, y_p_prob, average="weighted", multi_class="ovr")}')
print(f'linear+pca roc_auc_score = {roc_auc_score(Y_test_load_digits, y_p_pca_prob, average="weighted", multi_class="ovr")}')

linear roc_auc_score = 0.9980106704641046
linear+pca roc_auc_score = 0.9930071278977135


# Задание 2.

Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [115]:
def svd(X, components=3):
  U, s, W = np.linalg.svd(X)

  D = np.zeros_like(X, dtype=float)
  D[np.diag_indices(min(X.shape))] = s

  D = D[:, :components]
  W = W[:components, :]

  # исходная матрица
  X_ = U.dot(D.dot(W))
    
  print(f'Проверка разложения. Восстановление исходной матрицы.:\n {X_}\n')
  print(f'Матрица весов\n {W}\n')
  
  Z = U @ D
  print(f'Трансформированная матрица:\n {Z}\n')
  # return Z

In [116]:
A = np.array([[1, 2, 0], [0, 0, 5], [3, -4, 2], [1, 6, 5], [0, 1, 0]])

svd(A)

Проверка разложения. Восстановление исходной матрицы.:
 [[ 1.00000000e+00  2.00000000e+00  1.33720949e-15]
 [ 1.33806959e-16 -3.25169545e-15  5.00000000e+00]
 [ 3.00000000e+00 -4.00000000e+00  2.00000000e+00]
 [ 1.00000000e+00  6.00000000e+00  5.00000000e+00]
 [-3.55459915e-17  1.00000000e+00 -4.93664785e-18]]

Матрица весов
 [[ 0.07116451  0.71702467  0.69340553]
 [-0.36737824  0.66514082 -0.65009301]
 [-0.92734505 -0.20847855  0.31075368]]

Трансформированная матрица:
 [[ 1.50521386  0.9629034  -1.34430215]
 [ 3.46702765 -3.25046504  1.5537684 ]
 [-1.26779411 -5.06288401 -1.32661359]
 [ 7.8403402   0.37300164 -0.62444796]
 [ 0.71702467  0.66514082 -0.20847855]]

